In [8]:
import re
import os 
import sys 

import numpy as np
import matplotlib.pyplot as plt
import skimage
from skimage import io

from pathlib import Path
from tqdm.notebook import trange, tqdm
from joblib import Parallel, delayed
from skimage import exposure
import h5py
import pandas as pd
import scanpy as sc
import anndata as ad
import squidpy as sq
sc.settings.verbosity = 3

from matplotlib.pyplot import rc_context
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from functools import reduce
from matplotlib import cm, colors
import scanorama
import seaborn as sns 


In [9]:
p_dir = (Path().cwd().parents[0]).absolute()
data_dir = p_dir / 'data' 

In [10]:
%load_ext autoreload
%autoreload 2

module_path = str(p_dir / "src")

if module_path not in sys.path:
    sys.path.append(module_path)

import utils as my_utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
donors = [
    "LN Donor A",
    "LN Donor E",
    "INT Donor B",
    "INT Donor E",
    "TS Donor A",
    "TS Donor E",
]



# Read IMC Data

In [12]:
import matplotlib.patches as mpatches
from skimage.segmentation import mark_boundaries

donors = [
    "LN Donor A",
    "LN Donor E",
    "INT Donor B",
    "INT Donor E",
    "TS Donor A",
    "TS Donor E",
]


def get_imgs(file_path, name):
    f = h5py.File(file_path, "r")
    imgs = f[name]
    labels = list(f[name].attrs["labels"])
    return imgs, labels


def contrast_streching(img):
    p2, p98 = np.percentile(img, (1, 99))
    return exposure.rescale_intensity(img, in_range=(p2, p98))


# Read mask image
def get_masks(mask_folder):
    """
    Function to get all mask from mask forlder
    """
    # Read masks
    masks = {}

    for (dirpath, dirnames, filenames) in os.walk(mask_folder):
        for name in sorted(filenames):
            if "tif" in name:
                filename = os.path.join(dirpath, name)
                img = skimage.io.imread(filename)
                condition =  name.split(".")[0]
                masks[condition] = img
            else:
                continue
    return masks

def read_intensity_per_cell(img, mask):
    props = skimage.measure.regionprops_table(
        mask, img, properties=["label", "mean_intensity", "area"]
    )
    df_prop = pd.DataFrame(props)
    df_prop["mean_intensity"] = df_prop["mean_intensity"]
    df_prop.drop("area", axis=1, inplace=True)

    return df_prop

def get_img_subset(imgs, marker):
    imgs_subset = []
    for marker in markers:
        idx = labels.index(marker)
        imgs_subset.append(imgs[idx][:1000, :1000])
    return imgs_subset

In [13]:
markers = [
    "CD38",
    "Vimentin",
    "CD21",
    "BCL6",
    "ICOS1",
    "CD11b",
    'CXCR4',
    "CD11c",
    "FoxP3",
    "CD4",
    "CD138",
    "CXCR5",
    "CD20",
    "CD8",
    "C-Myc",
    "PD1",
    "CD83",
    "Ki67",
    "COL1",
    "CD3",
    "CD27",
    "EZH2",
    "H3K27me3",
]
df_all = []
centroids = []

In [14]:
# Format row, col
arrangement = {
    "LN Donor A": {
        1: [0, 1000],
        2: [0, 2000],
        3: [1000, 0],
        4: [1000, 1000],
        5: [1000, 2000],
        6: [1000, 3000],
        7: [2000, 0],
        8: [2000, 1000],
        9: [2000, 2000],
        10: [2000, 3000],
        11: [3000, 0],
        12: [3000, 1000],
        13: [3000, 2000],
        14: [3000, 3000],
        15: [4000, 1000],
        16: [4000, 2000],
    },
    "LN Donor E": {
        1: [1000, 0],
        2: [1000, 1000],
        3: [1000, 2000],
        4: [1000, 3000],
        5: [1000, 4000],
        6: [1000, 5000],
        7: [1000, 6000],
        8: [1000, 7000],
        9: [1000, 8000],
        10: [0, 0],
        11: [0, 1000],
        12: [0, 2000],
        13: [0, 3000],
        14: [0, 4000],
    },
    "INT Donor B": {
        1: [0, 0],
        2: [0, 1000],
        3: [1000, 0],
        4: [1000, 1000],
        5: [2000, 0],
        6: [2000, 1000],
        7: [2000, 2000],
        8: [2000, 3000],
        9: [3000, 0],
        10: [3000, 1000],
        11: [3000, 2000],
        12: [3000, 3000],
        13: [4000, 0],
        14: [4000, 1000],
        15: [4000, 2000],
        16: [4000, 3000],
        # 17: [5000, 0],
        # 18: [5000, 1000],
        # 19: [5000, 2000],
        # 20: [5000, 3000],
    },
    "INT Donor E": {
        1: [0, 0],
        2: [0, 1000],
        3: [0, 2000],
        4: [0, 3000],
        # 5: [0, 4000],
        6: [1000, 0],
        7: [1000, 1000],
        8: [1000, 2000],
        9: [1000, 3000],
        10: [1000, 4000],
        11: [2000, 3000],
        12: [2000, 4000],
        13: [3000, 3000],
        14: [3000, 4000],
        15: [4000, 3000],
        16: [4000, 4000],
    },
    "TS Donor A": {
        1: [0, 0],
        2: [0, 1000],
        3: [0, 2000],
        4: [0, 3000],
        5: [0, 4000],
        6: [0, 5000],
        7: [0, 6000],
        8: [1000, 0],
        9: [1000, 1000],
        10: [1000, 2000],
        11: [1000, 3000],
        12: [1000, 4000],
        13: [1000, 5000],
        14: [1000, 6000],
    },
    "TS Donor E": {
        1: [0, 0],
        2: [0, 1000],
        3: [0, 2000],
        4: [1000, 0],
        5: [1000, 1000],
        6: [1000, 2000],
        7: [2000, 0],
        8: [2000, 1000],
        9: [2000, 2000],
        10: [3000, 0],
        11: [3000, 1000],
        12: [3000, 2000],
        13: [4000, 0],
        14: [4000, 1000],
        15: [4000, 2000],
        16: [5000, 0],
        17: [5000, 1000],
        18: [5000, 2000],
    },
    "SP Donor A": {
        1: [0, 0],
        2: [0, 1000],
        3: [0, 2000],
        4: [0, 3000],
        5: [0, 4000],
        6: [1000, 0],
        7: [1000, 1000],
        8: [1000, 2000],
        9: [1000, 3000],
        10: [1000, 4000],
        11: [2000, 0],
        12: [2000, 1000],
        13: [2000, 2000],
        14: [2000, 3000],
        15: [2000, 4000],
        16: [3000, 0],
        17: [3000, 1000],
        18: [3000, 2000],
        19: [3000, 3000],
        20: [3000, 4000],
    },
}

In [15]:
for donor in donors:
    h5_data = p_dir / "data" / "h5_new" / f"{donor}.hdf5"
    df = pd.read_csv(data_dir / "metadata" / f"info_{donor}.csv")
    ROIs = df.ROI.unique()
    # masks = get_masks(data_dir / "masks_cp" / donor)
    masks = get_masks(data_dir / "masks" / donor)
    
    for roi in tqdm(ROIs, total=len(ROIs)):
        if roi not in arrangement[donor].keys():
            continue
        imgs, labels = get_imgs(h5_data, str(roi))
        mask = masks[str(roi)]

        imgs = get_img_subset(imgs, markers)

        df_appended_list = []
        for i, img in enumerate(imgs):
            df_prop = read_intensity_per_cell(img, mask)
            df_prop.columns = ["Cell_label", markers[i]]
            df_appended_list.append(df_prop)

        # Combine dataframe
        df_cell_intensity = reduce(
            lambda left, right: pd.merge(left, right, on=["Cell_label"]),
            df_appended_list,
        )
        df_cell_intensity["ROI"] = roi
        df_cell_intensity["Donor"] = donor
        df_all.append(df_cell_intensity)

        props = skimage.measure.regionprops_table(
            mask, properties=["label", "centroid"]
        )
        rows = props["centroid-0"] + arrangement[donor][roi][0]
        cols = props["centroid-1"] + arrangement[donor][roi][1]
        centroid = np.array(list(zip(cols, rows)))
        centroids.append(centroid)

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/14 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

In [16]:
centroids = np.concatenate(centroids)

In [17]:
df = pd.concat(df_all, ignore_index=True)
df.head()

,Cell_label,CD38,Vimentin,CD21,BCL6,ICOS1,CD11b,CXCR4,CD11c,FoxP3,...,PD1,CD83,Ki67,COL1,CD3,CD27,EZH2,H3K27me3,ROI,Donor
0,1,0.000000,17.400000,0.175000,0.0,0.00000,0.00000,0.0,1.025000,0.0,...,0.0,0.0,0.0,3.450000,0.000000,0.0,0.0,39.925000,1,LN Donor A
1,2,0.000000,25.460674,0.000000,0.0,0.00000,0.00000,0.0,0.898876,0.0,...,0.0,0.0,0.0,0.707865,0.000000,0.0,0.0,97.651685,1,LN Donor A
2,3,0.000000,6.357143,0.166667,0.0,0.47619,0.00000,0.0,0.976190,0.0,...,0.0,0.0,0.0,3.357143,0.000000,0.0,0.0,27.976190,1,LN Donor A
3,4,0.054545,25.454545,0.109091,0.0,0.00000,0.00000,0.0,1.436364,0.0,...,0.0,0.0,0.0,3.636364,0.000000,0.0,0.0,44.236364,1,LN Donor A
4,5,0.000000,35.765625,0.000000,0.0,0.09375,0.03125,0.0,1.156250,0.0,...,0.0,0.0,0.0,0.906250,2.171875,0.0,0.0,56.562500,1,LN Donor A


In [18]:
df_intensity = df.iloc[:, 1:-2]

In [19]:
df_intensity.head()

,CD38,Vimentin,CD21,BCL6,ICOS1,CD11b,CXCR4,CD11c,FoxP3,CD4,...,CD8,C-Myc,PD1,CD83,Ki67,COL1,CD3,CD27,EZH2,H3K27me3
0,0.000000,17.400000,0.175000,0.0,0.00000,0.00000,0.0,1.025000,0.0,0.000000,...,0.300000,1.137500,0.0,0.0,0.0,3.450000,0.000000,0.0,0.0,39.925000
1,0.000000,25.460674,0.000000,0.0,0.00000,0.00000,0.0,0.898876,0.0,0.000000,...,0.000000,0.887640,0.0,0.0,0.0,0.707865,0.000000,0.0,0.0,97.651685
2,0.000000,6.357143,0.166667,0.0,0.47619,0.00000,0.0,0.976190,0.0,0.285714,...,0.000000,1.119048,0.0,0.0,0.0,3.357143,0.000000,0.0,0.0,27.976190
3,0.054545,25.454545,0.109091,0.0,0.00000,0.00000,0.0,1.436364,0.0,0.000000,...,0.000000,1.290909,0.0,0.0,0.0,3.636364,0.000000,0.0,0.0,44.236364
4,0.000000,35.765625,0.000000,0.0,0.09375,0.03125,0.0,1.156250,0.0,1.828125,...,0.046875,2.218750,0.0,0.0,0.0,0.906250,2.171875,0.0,0.0,56.562500


In [20]:
# Create annData from dataframe
adata = sc.AnnData(df_intensity.values)
adata.var_names = df_intensity.columns.tolist()  # add variable name

# Add obs information
adata.obs["ROI"] = df.ROI.tolist()
adata.obs["Cell"] = df.Cell_label.tolist()
adata.obs["Dataset"] = df.Donor.tolist()
adata.obsm["spatial"] = centroids

C:\Users\thu71\AppData\Local\Temp\ipykernel_26184\58115827.py:2: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  adata = sc.AnnData(df_intensity.values)


In [21]:
adata_path = data_dir / "metadata" / f"combined_raw_12_12_new_mask.h5ad"
adata.write(adata_path)

In [22]:
df.to_csv(data_dir / "metadata" / f"combined_raw_12_12_new_mask.csv")

# Convert to FCS files

In [23]:
import flowkit as fk
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

In [24]:
df = pd.read_csv(data_dir / "metadata" / f"combined_raw_12_12_new_mask.csv", )
df = df.iloc[:,1:]

adata_path = data_dir / "metadata" / f"combined_raw_12_12_new_mask.h5ad"
adata = sc.read_h5ad(adata_path)
centroids = adata.obsm["spatial"]


In [25]:
import palettable
heatmap_cmp = palettable.cmocean.diverging.Balance_20.mpl_colormap

In [26]:
# Scale data
scaler = MinMaxScaler()

# Reset to get all samples index
df_2labels = df.reset_index()

# # Normalize per Tissue
# for donor in df_2labels['Donor'].unique():
#     df_subset = df_2labels[df_2labels.Donor == donor]
#     data = df_subset.iloc[:, 2:-2]
#     data_scaled = scaler.fit_transform(data)*(2**18)
#     # data_scaled = np.log(data_scaled+1)
#     # data_scaled = np.arcsinh(data_scaled)
#     df_2labels.loc[df_2labels.Donor == donor, 2:-2] = data_scaled

# Normalize all togther
data = df_2labels.iloc[:, 2:-2]
data_scaled = scaler.fit_transform(data)*(2**18)
# data_scaled = np.log(data_scaled+1)
# data_scaled = np.arcsinh(data_scaled)
df_2labels.loc[:, 2:-2] = data_scaled


# Add spatial info
df_2labels['X'] = centroids[:,1]
df_2labels['Y'] = centroids[:,0]


C:\Users\thu71\AppData\Local\Temp\ipykernel_26184\4154266611.py:21: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  df_2labels.loc[:, 2:-2] = data_scaled


In [27]:
for donor in df_2labels['Donor'].unique():
    df_subset = df_2labels[df_2labels.Donor == donor]
    df_subset.drop('Donor', inplace=True, axis=1)
    sample = fk.Sample(df_subset, cache_original_events=True)
    sample.original_filename = donor
    
    df_channels = sample.channels
    df_channels.pnr = np.max(df_subset).tolist()
    sample.channels = df_channels
    
    sample.export(f'{donor}.fcs', source='orig')

C:\Users\thu71\AppData\Local\Temp\ipykernel_26184\2099152042.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset.drop('Donor', inplace=True, axis=1)
C:\Users\thu71\Anaconda3\envs\scspamet\lib\site-packages\numpy\core\fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)
C:\Users\thu71\AppData\Local\Temp\ipykernel_26184\2099152042.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset.drop('Donor', inp

In [28]:
le = preprocessing.LabelEncoder()
df_2labels['Donor'] = le.fit_transform(df_2labels['Donor'])

sample = fk.Sample(df_2labels, cache_original_events=True)

df_channels = sample.channels
df_channels.pnr = np.max(df_2labels).tolist()
sample.channels = df_channels

C:\Users\thu71\Anaconda3\envs\scspamet\lib\site-packages\numpy\core\fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


In [29]:
sample.export('lymphoid.fcs', source='orig')